NYC Skyline

There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market.

The goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [16]:
# loading the packages neede
import numpy as np
import pandas as pd

In [17]:

# upload the data files
prices = pd.read_csv("data/airbnb_price.csv")

In [18]:
xls = pd.ExcelFile("data/airbnb_room_type.xlsx")
room_types = xls.parse(0)

In [19]:
reviews = pd.read_csv("data/airbnb_last_review.tsv", sep= "\t")

In [20]:
# Unify the price column and convert tu numeric type
prices["price"] = prices["price"].str.replace(' dollars', '')
prices["price"] = pd.to_numeric(prices["price"])
prices["price"]

0        225
1         89
2        200
3         79
4        150
        ... 
25204    129
25205     45
25206    235
25207    100
25208     30
Name: price, Length: 25209, dtype: int64

In [21]:
# explore the prices dataset
prices.describe()

,listing_id,price
count,2.520900e+04,25209.000000
mean,2.068922e+07,141.777936
std,1.102928e+07,147.349137
min,2.595000e+03,0.000000
25%,1.202273e+07,69.000000
50%,2.234391e+07,105.000000
75%,3.037669e+07,175.000000
max,3.645581e+07,7500.000000


In [22]:
# find the avarage price
free_listings = prices["price"] == 0
prices = prices.loc[~free_listings]
avg_price = round(prices['price'].mean(), 2)
avg_price


141.82

In [23]:
# avarage price for month
prices["price_per_month"] = prices['price']*(365/12)
average_price_per_month = round(prices["price_per_month"].mean(), 2)
difference = round((average_price_per_month - 3100), 2)
difference

1213.61

In [27]:
# categorize the room types column and count how many of each of them
room_types['room_type'] = room_types['room_type'].str.lower()
room_types['room_type'] = room_types['room_type'].astype('category')
room_frequencies = room_types['room_type'].value_counts()
room_frequencies

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64

In [28]:
# find the first and last reviews 
reviews['last_review'] = pd.to_datetime(reviews['last_review'])
first_reviewed = reviews['last_review'].dt.date.min()
last_reviewed = reviews['last_review'].dt.date.max()

In [35]:
# join the 3 datasets and confirm the action
rooms_and_prices = prices.merge(room_types, how='outer', on='listing_id')
airbnb_merged = rooms_and_prices.merge(reviews, how='outer', on='listing_id')
airbnb_merged.dropna(inplace=True)
airbnb_merged.duplicated().sum()


0

In [37]:
# look for descriptive statistic for each borough
airbnb_merged['borough'] = airbnb_merged["nbhood_full"].str.partition(",")[0]

boroughs = airbnb_merged.groupby('borough')['price'].agg(['sum', 'mean', 'median', 'count'])
boroughs = boroughs.round(2).sort_values("mean", ascending=False)
boroughs

,sum,mean,median,count
borough,,,,
Manhattan,1898417.0,184.04,149.0,10315
Brooklyn,1275250.0,122.02,95.0,10451
Queens,320715.0,92.83,70.0,3455
Staten Island,22974.0,86.04,71.0,267
Bronx,55156.0,79.25,65.0,696


In [40]:
#categorize each borough by price_ranges
label_names = ["Budget", "Average", "Expensive", "Extravagant"]
ranges = [0, 69, 175, 350, np.inf]

airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

prices_by_borough = airbnb_merged.groupby(['borough', 'price_range'])['price_range'].count()

prices_by_borough

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64

In [42]:
# summerize the analysis above
airbnb_analysis = {'avg_price':avg_price, 'average_price_per_month':average_price_per_month, 'difference':difference, 'room_frequencies':room_frequencies, 'first_reviewed':first_reviewed, 'last_reviewed':last_reviewed, 'prices_by_borough':prices_by_borough}

print(airbnb_analysis)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  